In [1]:
import pytorch_lightning as pl
import pandas as pd
import numpy as np
import torch
from rdkit.Chem.rdmolfiles import MolFromSmiles
from rdkit.Chem.rdmolops import RDKFingerprint
from torch import nn

from molNet.dataloader.base_loader import GeneratorDataLoader, GeneratorDataset, df_to_generator
from molNet.dataloader.molecule_loader import MoleculeGraphFromGeneratorLoader, SmilesFromGeneratorLoader, \
    MoleculeFromGeneratorLoader, MoleculeLoaderMixin, MoleculeGraphLoaderMixin, PytorchGeomMolLoaderMixin, \
    PytorchGeomMolGraphFromGeneratorLoader, PytorchGeomMolGraphGenerator, SmilesfromDfGenerator
from molNet.featurizer.atom_featurizer import default_atom_featurizer
from molNet.nn.models.graph_convolution import GraphFingerprint,ExtendedGrapFingerprint

In [2]:
#df = pd.read_csv("../../datasets/chembl_27_chemreps.txt.gz",sep="\t")
df = pd.read_csv("../../datasets/delaney-processed.csv")

In [3]:
smiles_col="smiles"
#smiles_col="canonical_smiles"
fp_size=100
df['fp'] = None #= df[smiles_col].apply(lambda s:np.array(RDKFingerprint(MolFromSmiles(s),fpSize=fp_size)))

In [7]:
loader = PytorchGeomMolGraphFromGeneratorLoader(
    *[PytorchGeomMolGraphGenerator(atom_featurizer=default_atom_featurizer,generator=g)
     for g in
        df_to_generator(df,generator_class=SmilesfromDfGenerator,smiles_col=smiles_col)],
    batch_size=64)

model = ExtendedGrapFingerprint(len(default_atom_featurizer),len_additional_data=8,fully_connected_layer=[20],fingerprint_size=None)

loader.setup()
for b in loader.test_dataloader():
    print(b)
    print(model(b)[0].shape)
    break

Batch(batch=[1711], edge_index=[2, 3446], graph_features=[64, 8], graph_features_titles=[64], string_data=[64], string_data_titles=[64], x=[1711, 141], y=[0])
torch.Size([20])


In [8]:
model

ExtendedGrapFingerprint(
  (gnn): ModuleDict(
    (0_gc): GCNConv(141, 141)
    (1_gc): GCNConv(141, 141)
    (2_gc): GCNConv(141, 141)
    (3_gc): GCNConv(141, 141)
    (4_gc): GCNConv(141, 141)
    (5_gc): GCNConv(141, 141)
  )
  (pooling): MergedPooling(
    (pooling_layer): ModuleDict(
      (0_pooling_weight_sum): PoolWeightedSum(
        (weighting_of_nodes): Sequential(
          (0): Linear(in_features=141, out_features=1, bias=True)
          (1): Sigmoid()
        )
      )
      (1_pooling_max): PoolMax()
    )
  )
  (graph_out): ModuleDict(
    (0_post_linear): Linear(in_features=290, out_features=20, bias=True)
  )
  (loss): MultiLabelSoftMarginLoss()
)